In [ ]:
%pip install -qqq -U bitsandbytes==0.39.0
%pip install -qqq torch==2.0.1
%pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
%pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
%pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
%pip install -qqq -U datasets==2.12.0
%pip install -qqq -U loralib==0.1.1
%pip install -qqq -U einops==0.6.1

%pip install scipy

In [27]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig

#model_name = "EleutherAI/gpt-neox-20b"
#model_name = "THUDM/chatglm-6b"
model_name = "tiiuae/falcon-7b-instruct"

#Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

import os 
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:64'

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [28]:
#model = AutoModel.from_pretrained(model_name, quantization_config=quant_config, trust_remote_code=True, device_map={"":0})
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, trust_remote_code=True, device_map={"":0})

model.gradient_checkpointing_enable()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ming/anaconda3/envs/llm_lora1/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda121.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-12/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 121
CUDA SETUP: Loading binary /home/ming/anaconda3/envs/llm_lora1/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda121.so...


/home/ming/anaconda3/envs/llm_lora1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/ming/anaconda3/envs/llm_lora1 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/ming/anaconda3/envs/llm_lora1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda-12.0/lib64'), PosixPath('}')}
  warn(msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.38s/it]


In [29]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [25]:
import json

data = []
state = 0
with open("data/training_1") as file:
    lines = file.readlines()
    msg = ''
    output = ''
    for i in range(0, len(lines)):
        if lines[i].startswith('Example '):
            if msg != '':
                data.append({'msg': msg, 'output': output})
            msg = ''
            output = ''
            continue
        elif lines[i].startswith('msg ='):
            state = 2
        elif lines[i].startswith('output ='):
            state = 3
            
        if state == 2:
            msg = msg + lines[i].lstrip('msg =').replace('\"', '').strip()
        elif state == 3:
            output = output + lines[i].lstrip('output =').strip()


with open("data/train_dataset_1.json", "w") as f:
    json.dump(data, f)

data

[{'msg': 'Hi there! Do you have any recommendations for a new laptop?',
  'output': '[{"intent_name": "Intent_greeting", "answer": "Hello! How can I assist you today?"},{"intent_name": "Intent_prod", "tool_query": "laptop"}]'},
 {'msg': 'What are the current deals and promotions available?',
  'output': '[{"intent_name": "Intent_cs", "tool_query": "current deals and promotions"}]'},
 {'msg': 'I need help with a technical issue on my smartphone.',
  'output': '[{"intent_name": "Intent_cs", "tool_query": "smartphone technical issue"}]'},
 {'msg': 'Can you provide information on the warranty for your products?',
  'output': '[{"intent_name": "Intent_cs", "tool_query": "product warranty"}]'},
 {'msg': 'Hello! How long does the shipping usually take?',
  'output': '[{"intent_name": "Intent_greeting", "answer": "Hello! How can I assist you today?"},{"intent_name": "Intent_cs", "tool_query": "shipping duration"}]'},
 {'msg': 'I want to check the status of my order.',
  'output': '[{"intent_na

In [26]:
from datasets import load_dataset

data = load_dataset("json", data_files="data/train_dataset_1.json")
data
data["train"][:10]


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 876.37it/s]


Dataset json downloaded and prepared to /home/ming/.cache/huggingface/datasets/json/default-056784959c069a4b/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 485.11it/s]


{'output': ['[{"intent_name": "Intent_greeting", "answer": "Hello! How can I assist you today?"},{"intent_name": "Intent_prod", "tool_query": "laptop"}]',
  '[{"intent_name": "Intent_cs", "tool_query": "current deals and promotions"}]',
  '[{"intent_name": "Intent_cs", "tool_query": "smartphone technical issue"}]',
  '[{"intent_name": "Intent_cs", "tool_query": "product warranty"}]',
  '[{"intent_name": "Intent_greeting", "answer": "Hello! How can I assist you today?"},{"intent_name": "Intent_cs", "tool_query": "shipping duration"}]',
  '[{"intent_name": "Intent_cs", "tool_query": "order status"}]',
  '[{"intent_name": "Intent_prod", "tool_query": "budget-friendly headphones"}]',
  '[{"intent_name": "Intent_cs", "tool_query": "package tracking"}]',
  '[{"intent_name": "Intent_cs", "tool_query": "account login issue"}]',
  '[{"intent_name": "Intent_greeting", "answer": "Hello! How can I assist you today?"},{"intent_name": "Intent_cs", "tool_query": "refund assistance"}]'],
 'msg': ['Hi 

In [30]:
from string import Template

with open('prompts/prompt-f1', 'r') as file:
    template = Template(file.read())

def gen_prompt(rec) -> str:
    return template.substitute(msg=rec["msg"], output=rec["output"]).strip()

def gen_and_tokenize_prompt(rec):
    prompt = gen_prompt(rec)
    print(prompt + "\n")
    encoding = tokenizer(prompt, padding=True, truncation=True)
    return encoding

In [31]:
train_data = data["train"].shuffle().map(gen_and_tokenize_prompt)
train_data


You are a store chatbot that can be programmed by python. You now execute the following code:
The code section starts with <code> and end with </code>
Please assume that all functions without definitions are already implemented as the comments suggests.    

<code>
import json
from enum import Enum

class IntentName(Enum):
    INTENT_greeting = "Intent_greeting" 
    INTENT_cs = "Intent_cs"
    INTENT_prod = "Intent_prod"
    INTENT_default = "Intent_default"

class Intent:
    def __init__(self, intent_name: IntentName, query: str = None, tool_query: str = None, answer: str = None):
        self.intent_name = intent_name
        self.query = query
        self.tool_query = tool_query
        self.answer = answer

def process(msg: str) -> str:
    intents = []
    intent_name = getIntentName(msg)
    
    if intent_name == IntentName.INTENT_prod:  # has an intent to find a single product
        # extract a tool_query to search for product database
        tool_query = extractProdQuery

Dataset({
    features: ['output', 'msg', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 49
})

In [32]:
import transformers

MODEL_OUTPUT_DIR = "experiments"

%load_ext tensorboard
%tensorboard --host "0.0.0.0" --logdir experiments/runs

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=MODEL_OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.357100
2,1.361700
3,1.350900
4,1.354900
5,1.307000
6,1.316700
7,1.263500
8,1.242700
9,1.181300
10,1.130200


TrainOutput(global_step=80, training_loss=0.31761557994177564, metrics={'train_runtime': 774.1522, 'train_samples_per_second': 0.413, 'train_steps_per_second': 0.103, 'total_flos': 4251526121233152.0, 'train_loss': 0.31761557994177564, 'epoch': 6.53})

In [33]:
model.save_pretrained("experiments/model-1")

In [34]:
from peft import PeftConfig
from peft import PeftModel

PEFT_MODEL = "experiments/model-1"
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, return_dict=True, quantization_config=quant_config, device_map={"":0}, trust_remote_code=True)
model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.82s/it]


In [37]:
from string import Template

def bot_answer(model, question):    
    gen_config = model.generation_config
    gen_config.max_new_tokens = 200
    gen_config.temperature = 0.7
    gen_config.top_p = 0.7
    gen_config.num_return_sequences = 1
    gen_config.pad_token_id = tokenizer.eos_token_id
    gen_config.eos_token_id = tokenizer.eos_token_id

    device = "cuda:0"
    prompt = template.substitute(msg=question, output="").strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        out = model.generate(
            input_ids = encoding.input_ids,
            use_cache=True,
            do_sample=True,
            num_beams=1,
            num_beam_groups=1,
            generation_config = gen_config
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [41]:
%%time
print(bot_answer(model, "Hi, what is return policy on computer laptops?") + "\n")


You are a store chatbot that can be programmed by python. You now execute the following code:
The code section starts with <code> and end with </code>
Please assume that all functions without definitions are already implemented as the comments suggests.    

<code>
import json
from enum import Enum

class IntentName(Enum):
    INTENT_greeting = "Intent_greeting" 
    INTENT_cs = "Intent_cs"
    INTENT_prod = "Intent_prod"
    INTENT_default = "Intent_default"

class Intent:
    def __init__(self, intent_name: IntentName, query: str = None, tool_query: str = None, answer: str = None):
        self.intent_name = intent_name
        self.query = query
        self.tool_query = tool_query
        self.answer = answer

def process(msg: str) -> str:
    intents = []
    intent_name = getIntentName(msg)
    
    if intent_name == IntentName.INTENT_prod:  # has an intent to find a single product
        # extract a tool_query to search for product database
        tool_query = extractProdQuery

In [39]:
%%time
print(bot_answer(model, "How to book flight from SEA to TPA?") + "\n")

You are a store chatbot that can be programmed by python. You now execute the following code:
The code section starts with <code> and end with </code>
Please assume that all functions without definitions are already implemented as the comments suggests.    

<code>
import json
from enum import Enum

class IntentName(Enum):
    INTENT_greeting = "Intent_greeting" 
    INTENT_cs = "Intent_cs"
    INTENT_prod = "Intent_prod"
    INTENT_default = "Intent_default"

class Intent:
    def __init__(self, intent_name: IntentName, query: str = None, tool_query: str = None, answer: str = None):
        self.intent_name = intent_name
        self.query = query
        self.tool_query = tool_query
        self.answer = answer

def process(msg: str) -> str:
    intents = []
    intent_name = getIntentName(msg)
    
    if intent_name == IntentName.INTENT_prod:  # has an intent to find a single product
        # extract a tool_query to search for product database
        tool_query = extractProdQuery